In [12]:
import sys,os,subprocess,glob
import pandas as pd
import numpy as np
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
from Bio import SeqIO
import urllib.request

In [14]:
def fetch_sequence_db(name='card'):
    """get sequences"""
    
    path = '.'
    if name == 'card':
        url = 'https://github.com/tseemann/abricate/raw/master/db/card/sequences'
    filename = os.path.join(path,"%s.fa" %name)
    if not os.path.exists(filename):
        urllib.request.urlretrieve(url, filename)
    return

fetch_sequence_db()

In [ ]:
seqs = []
for s in SeqIO.parse('/local/abricate/db/card/sequences','fasta'):
    try:
        seqs.append(s)
    except:
        pass 


In [4]:
def blast_card(ident=90,coverage=.75):
    """blast card seqs"""

    cardseqs = list(SeqIO.parse('/local/abricate/db/card/sequences','fasta'))
    #bl = tools.local_blast(qfile, 'scaffolds.fasta', ident=ident, params='-e 10 -a 2 -S 1')
    bl = tools.blast_sequences('scaffolds.fasta',cardseqs,maxseqs=100,evalue=.1,cmd='blastn',show_cmd=True)
    print (bl[:5])
    bl['qlength'] = bl.sequence.str.len()
    bl['coverage'] = bl.length/bl.qlength
    bl = bl[bl.coverage>coverage]
    bl = bl[bl.pident>ident]
    bl['id'] = bl.sseqid.apply(lambda x: x.split('~')[0],1)
    bl['contig'] = bl.sseqid.apply(lambda x: x.split('~')[1],1)
    bl['gene'] = bl['qseqid'].apply(lambda x: x.split('~~~')[1],1)
    #print (bl)
    cols = ['qseqid','sseqid','pident','sstart','send','coverage','contig','gene','id']
    bl = bl[cols]
    return bl

In [ ]:
blast_card()

In [ ]:
def make_blast_database(filenames):
    """make blast dbs"""

    rec=[]
    for n in filenames:
        seqs = list(SeqIO.parse('scaffolds/%s.fa' %n,'fasta'))
        for s in seqs:
            s.id = n + '~' + s.id
        rec.extend(seqs)
    ref = list(SeqIO.parse('genomes/ecoli_k12.fa','fasta'))
    ref[0].id = 'ecoli_k12~1'
    rec.extend(ref)
    SeqIO.write(rec, 'scaffolds.fasta', 'fasta')
    cmd = 'makeblastdb -dbtype nucl -in scaffolds.fasta'
    
    